# Logical Transduction

Agentics objects are capable of performing logical transduction between their states by using the provied LLMs. The logical transduction operator << can be applied between any two agentics of any type.

Transduction is done between a source and a target AG when connected by the << operator.

The follwing example transduces a Question into an Answer. 

In [ ]:
from agentics import Agentics as AG
from pydantic import BaseModel
from typing import Optional
from agentics.core.llm_connections import get_llm_provider


## Define target and source types
class Answer(BaseModel):
    answer: Optional[str] = None
    justification: Optional[str] = None
    confidence: Optional[float] = None

class Question(BaseModel):
    question :Optional[str] = None


## Instantiate the source AG with a question
source = AG(atype=Question,
            llm=get_llm_provider(), ## You can choose between "openai" (i.e. get_llm_provider("openai")), "watsonx", "gemini", "vllm_crewai"
                                    ## set verbose to true to see the internal agents log. This is optional.     
            states=[Question(question="What is the capital of Italy?")])

## Instantiate the target AG with a target type. No instances are needed for zero shot transduction
target= AG(atype=Answer,  ## You can choose between "openai", "watsonx", "gemini", "vllm_crewai"
           verbose_agent=True)   ## set verbose to true to see the internal agents log 

# Execute logical transduction by using the << operator between source and target AG
answer = await (target << source) ## Note that << operator is asyncronus and the results should be awaited

# Print the results of the transduction
print(answer.pretty_print()) ## Note that confidence is a float number, while other fields are strings

2025-09-02 12:38:54.956 | DEBUG    | agentics.core.llm_connections:<module>:89 - AGENTICS is connecting to the following LLM API providers:
2025-09-02 12:38:54.957 | DEBUG    | agentics.core.llm_connections:<module>:92 - 0 - WatsonX
2025-09-02 12:38:54.957 | DEBUG    | agentics.core.llm_connections:<module>:97 - 1 - Gemini
2025-09-02 12:38:54.957 | DEBUG    | agentics.core.llm_connections:<module>:101 - 2 - OpenAI
2025-09-02 12:38:54.957 | DEBUG    | agentics.core.llm_connections:<module>:103 - Please add API keys in .env file to add or disconnect providers.
2025-09-02 12:38:54.964 | DEBUG    | agentics.core.llm_connections:get_llm_provider:28 - No LLM provider specified. Using the first available provider.
2025-09-02 12:38:54.965 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'


NameError: name 'ollama_llm' is not defined

In agentics, lists of strings can be used as sources instead of AG. Those are provided as input for the transduction.

In [ ]:
answer = await (AG(atype=Answer) << ["Where is Paris?"]) 
print(answer.pretty_print())

2025-09-02 12:37:53.303 | DEBUG    | agentics.core.agentics:__lshift__:545 - Transduction from input texts ['Where is Paris?'] to <class 'pydantic._internal._model_construction.ModelMetaclass'> in progress. This might take a while
2025-09-02 12:37:53.304 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-02 12:37:54.495 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 1 states in 1.190882921218872 seconds
2025-09-02 12:37:54.496 | DEBUG    | agentics.core.agentics:__lshift__:700 - 1 states processed in 0.059544146060943604 seconds average per state ...


Atype : <class '__main__.Answer'>
answer: Paris is in France
justification: Geographical knowledge
confidence: 0.99




## Asyncronous Transduction

When the source AG has more than one state, the << operator perform asyncronous transfuction for each state to the target type. Transductions are executed in batches.

In [ ]:
target = AG(atype=Answer, 
            batch_size = 4, # This will process 2 questions per batch
            verbose_transduction = True, # Set to verbose to see transduction timings and other logs
            transduction_logs_path = "/tmp/answers.jsonl") # Optionally write longs of transductions on the specified path
questions= ["Where is Paris?", 
            "Who is Alberto Sordi", 
            "When will climate change be irreverible?",
            "Who is the best Jeopardy player?"]

answers = await (target << questions)

print(answer.pretty_print())

target.batch_size = 4  ## Processing 4 questions in parallel often results in
                                    ## lower execution time
                                    ## if supported by LLM backend

answers = await (target << questions)

2025-09-02 12:37:54.506 | DEBUG    | agentics.core.agentics:__lshift__:545 - Transduction from input texts ['Where is Paris?', 'Who is Alberto Sordi', 'When will climate change be irreverible?', 'Who is the best Jeopardy player?'] to <class 'pydantic._internal._model_construction.ModelMetaclass'> in progress. This might take a while
2025-09-02 12:37:54.506 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-02 12:37:56.793 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 4 states in 2.286468029022217 seconds
2025-09-02 12:37:56.795 | DEBUG    | agentics.core.agentics:__lshift__:700 - 4 states processed in 0.5716170072555542 seconds average per state ...
2025-09-02 12:37:56.796 | DEBUG    | agentics.core.agentics:__lshift__:545 - Transduction from input texts ['Where is Paris?', 'Who is Alberto Sordi', 'When will climate change be irreverible?', 'Who is the best Jeopardy

Atype : <class '__main__.Answer'>
answer: Paris is in France
justification: Geographical knowledge
confidence: 0.99




2025-09-02 12:38:00.577 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 4 states in 3.780372142791748 seconds
2025-09-02 12:38:00.577 | DEBUG    | agentics.core.agentics:__lshift__:700 - 4 states processed in 0.945093035697937 seconds average per state ...


## Self Transduction

Self transduction is a method of AGs that enables async execution of transductions between slots of the same object.

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional

## Define the Pydantic type
class Movie(BaseModel):
    movie_name: Optional[str] = None ## Note that fields name should match the column name in the input csv
    genre: Optional[str] = None
    description: Optional[str]=None
    tweet: Optional[str]=Field(None,description="Generate a Tweet to advertise the movie")

movies = AG.from_csv( "data/movies.csv", atype=Movie)
movies.verbose_transduction = True
movies.llm=get_llm_provider("watsonx") ## You can choose between "openai", "watsonx", "gemini", "vllm_crewai"
movies.states = movies.states[:100] ## Reduce the number of input movies to 10

movies_with_tweets = await movies.self_transduction(

    ["movie_name", "genre", "description"], ## source fields
    ["tweet"],                              ## target fields
    
    ## Note that instruction are only needed when the  relation between source and target type 
    # is not innediately clear and need to be further specified or disambiguated.
    instructions="Generate a tweet to advertise the release of the input movie")
    
print(movies_with_tweets.pretty_print())

2025-09-02 12:38:00.585 | DEBUG    | agentics.core.agentics:from_csv:312 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-02 12:38:00.588 | DEBUG    | agentics.core.llm_connections:get_llm_provider:37 - Using specified LLM provider: watsonx
2025-09-02 12:38:00.594 | DEBUG    | agentics.core.agentics:__lshift__:518 - Executing task: Generate a tweet to advertise the release of the input movie
100 states will be transduced
2025-09-02 12:38:00.595 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-02 12:38:03.291 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 20 states in 2.69573712348938 seconds
2025-09-02 12:38:03.292 | DEBUG    | agentics.core.agentics:__lshift__:700 - 20 states processed in 0.134786856174469 seconds average per state ...
2025-09-02 12:38:06.115 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 20 states in 2.822558879

Atype : <class '__main__.Movie'>
movie_name: The Shawshank Redemption
genre: Drama, Crime
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.
tweet: 'Get ready to experience hope and redemption! ''The Shawshank Redemption'',
  a gripping Drama/Crime film, is now out! Imprisoned for a crime he didn''t commit,
  Andy Dufresne''s journey to freedom will leave you inspired. Don''t miss it! #TheShawshankRedemption
  #NewRelease'

movie_name: The Godfather
genre: Drama, Crime
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patriarch, Vito Co

As an alternative, self transduction can be encoded using logical transduction algebra which uses the AG() notation to rebind the original data into AGs of the requested subtypes. Learn more about atype manipulation in agentics [here](link)

In [ ]:

movies = AG.from_csv( "data/movies.csv", atype=Movie)
tweets = await (AG(atype=movies("tweet").atype, 
                   instructions="Generate a tweet to advertise the release of the input movie") 
                << movies("movie_name","genre","description" ))
print(tweets.pretty_print())    ## Note that differently from self transduction the output tweets 
                                ##has only the tweet field, whereas self transduction preserves 
                                ## the original source data

2025-09-02 12:38:12.970 | DEBUG    | agentics.core.agentics:from_csv:312 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-02 12:38:12.973 | DEBUG    | agentics.core.agentics:__lshift__:518 - Executing task: Generate a tweet to advertise the release of the input movie
100 states will be transduced
2025-09-02 12:38:12.974 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-02 12:38:15.487 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 20 states in 2.5123982429504395 seconds
2025-09-02 12:38:15.488 | DEBUG    | agentics.core.agentics:__lshift__:700 - 20 states processed in 0.12561991214752197 seconds average per state ...
2025-09-02 12:38:18.019 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 20 states in 2.5308339595794678 seconds
2025-09-02 12:38:18.020 | DEBUG    | agentics.core.agentics:__lshift__:700 - 40 states processed in 0.12654

Atype : <class 'agentics.core.agentics.tweet'>
tweet: 'Get ready to experience hope and redemption! ''The Shawshank Redemption'',
  a gripping Drama/Crime film, is now out! Imprisoned for a crime he didn''t commit,
  Andy Dufresne''s journey to freedom will leave you inspired. Don''t miss it! #TheShawshankRedemption
  #NewRelease #MustWatch'

tweet: 'Get ready for a cinematic masterpiece! ''The Godfather'', a gripping Drama
  and Crime film, hits the screens! Spanning 1945 to 1955, this epic tale of the Corleone
  crime family''s bloody revenge will leave you on the edge of your seat. Don''t miss
  out! #TheGodfather #MovieRelease'

tweet: 'Get ready for a cinematic masterpiece! ''The Godfather Part II'' is now out!
  This Drama, Crime film tells the story of the Corleone crime family''s saga, from
  young Vito''s growth in Sicily and 1910s New York to Michael''s attempts to expand
  the family business into Las Vegas, Hollywood, and Cuba. #TheGodfatherPartII #Drama
  #Crime'

tweet: '

## Few Shots Transduction

Few shots examples can be provided for transduction by adding instances of the target instances in correspondance to their sources . Those will be used by the LLM to infer by analogy all the Null instances of the target type.


In [ ]:
movies = AG.from_csv("data/movies.csv")
movies.states=movies.states[:20]
## Note that obly the first 9 movies have categories
for i, movie in enumerate(movies): print(f"{i}: {movie.genre}")
## predicting new genre from given examples
all_genres=await (movies("genre")<<movies("movie_name",  "description"))
print(all_genres.pretty_print())
## few shots examples can also be used in self transfuction
movies_with_genre = await movies.self_transduction(["movie_name",  "description"],["genre"])
print(movies_with_genre.pretty_print())


2025-09-02 12:38:25.198 | DEBUG    | agentics.core.agentics:__lshift__:518 - Executing task: Generate an object of the specified type from the following input.
20 states will be transduced
2025-09-02 12:38:25.198 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>


0: Drama, Crime
1: Drama, Crime
2: Drama, Crime
3: Drama, History, War
4: Drama
5: Animation, Family, Fantasy
6: Drama, Action, Crime, Thriller
7: Comedy, Drama, Romance
8: Fantasy, Drama, Crime
9: 
10: 
11: 
12: 
13: 
14: 
15: 
16: 
17: 
18: 
19: 


2025-09-02 12:38:26.895 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 20 states in 1.6968047618865967 seconds
2025-09-02 12:38:26.896 | DEBUG    | agentics.core.agentics:__lshift__:700 - 20 states processed in 0.08484023809432983 seconds average per state ...
2025-09-02 12:38:26.898 | DEBUG    | agentics.core.agentics:__lshift__:518 - Executing task: Generate an object of the specified type from the following input.
20 states will be transduced
2025-09-02 12:38:26.899 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>


Atype : <class 'agentics.core.agentics.genre'>
genre: Drama, Crime

genre: Drama, Crime

genre: Drama, Crime

genre: Drama, History, War

genre: Drama

genre: Animation, Family, Fantasy

genre: Drama, Action, Crime, Thriller

genre: Comedy, Drama, Romance

genre: Fantasy, Drama, Crime

genre: Drama, Thriller

genre: Crime, Drama

genre: Animation, Fantasy, Romance

genre: Adventure, Fantasy

genre: Drama, Romance

genre: Western, Drama, Adventure

genre: Action, Adventure, Drama

genre: Drama, Crime

genre: Adventure, Science Fiction

genre: Drama, War

genre: Drama, Comedy




2025-09-02 12:38:28.057 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 20 states in 1.1577129364013672 seconds
2025-09-02 12:38:28.057 | DEBUG    | agentics.core.agentics:__lshift__:700 - 20 states processed in 0.05788564682006836 seconds average per state ...


Atype : <class 'agentics.core.utils.AType#movie_name:genre:descriptionOptional'>
movie_name: The Shawshank Redemption
genre: Drama, Crime
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.

movie_name: The Godfather
genre: Drama, Crime
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patriarch, Vito Corleone barely
  survives an attempt on his life, his youngest son, Michael steps in to take care
  of the would-be killers, launching a campaign of bloody revenge.

movie_name: The Godfather Part II
genre: Drama, Crime
description: In the